# Topic Modeling

In [1]:
from gensim import corpora, models
from normalization import normalize_corpus
import numpy as np

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [2]:
toy_corpus = ["The fox jumps over the dog",
"The fox is very clever and quick",
"The dog is slow and lazy",
"The cat is smarter than the fox and the dog",
"Python is an excellent programming language",
"Java and Ruby are other programming languages",
"Python and Java are very popular programming languages",
"Python programs are smaller than Java programs"]

# LATENT SEMANTIC INDEXING

In [3]:
# LSI topic model
norm_tokenized_corpus = normalize_corpus(toy_corpus, tokenize=True)
norm_tokenized_corpus

dictionary = corpora.Dictionary(norm_tokenized_corpus)
print (dictionary.token2id)

corpus = [dictionary.doc2bow(text) for text in norm_tokenized_corpus]
corpus

tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

total_topics = 2

lsi = models.LsiModel(corpus_tfidf, 
                      id2word=dictionary, 
                      num_topics=total_topics)
                      
for index, topic in lsi.print_topics(total_topics):
    print ('Topic #'+str(index+1))
    print (topic)
    print ()             
    


{'dog': 0, 'fox': 1, 'jump': 2, 'clever': 3, 'quick': 4, 'lazy': 5, 'slow': 6, 'cat': 7, 'smarter': 8, 'excellent': 9, 'language': 10, 'programming': 11, 'python': 12, 'java': 13, 'ruby': 14, 'popular': 15, 'program': 16, 'small': 17}
Topic #1
-0.459*"programming" + -0.459*"language" + -0.344*"java" + -0.344*"python" + -0.336*"popular" + -0.318*"excellent" + -0.318*"ruby" + -0.148*"program" + -0.074*"small" + 0.000*"quick"

Topic #2
-0.459*"dog" + -0.459*"fox" + -0.444*"jump" + -0.322*"cat" + -0.322*"smarter" + -0.208*"lazy" + -0.208*"slow" + -0.208*"clever" + -0.208*"quick" + -0.000*"popular"



In [4]:

def print_topics_gensim(topic_model, total_topics=1,
                        weight_threshold=0.0001,
                        display_weights=False,
                        num_terms=None):
    
    for index in range(total_topics):
        topic = topic_model.show_topic(index)
        topic = [(word, round(wt,2)) 
                 for word, wt in topic 
                 if abs(wt) >= weight_threshold]
        if display_weights:
            print ('Topic #'+str(index+1)+' with weights')
            print (topic[:num_terms] if num_terms else topic)
        else:
            print ('Topic #'+str(index+1)+' without weights')
            tw = [term for term, wt in topic]
            print (tw[:num_terms] if num_terms else tw)
        print ()
    


In [5]:
# print topics without weights
print_topics_gensim(topic_model=lsi,
                    total_topics=total_topics,
                    num_terms=5,
                    display_weights=True)

Topic #1 with weights
[('programming', -0.46), ('language', -0.46), ('java', -0.34), ('python', -0.34), ('popular', -0.34)]

Topic #2 with weights
[('dog', -0.46), ('fox', -0.46), ('jump', -0.44), ('cat', -0.32), ('smarter', -0.32)]



# LSI custom built topic model   

In [7]:
from utils import build_feature_matrix, low_rank_svd

norm_corpus = normalize_corpus(toy_corpus)

vectorizer, tfidf_matrix = build_feature_matrix(norm_corpus, 
                                    feature_type='tfidf')
td_matrix = tfidf_matrix.transpose()
                              
td_matrix = td_matrix.multiply(td_matrix > 0)

total_topics = 2
feature_names = vectorizer.get_feature_names()

u, s, vt = low_rank_svd(td_matrix, singular_count=total_topics)
weights = u.transpose() * s[:, None]



In [8]:

def get_topics_terms_weights(weights, feature_names):
    feature_names = np.array(feature_names)
    sorted_indices = np.array([list(row[::-1]) 
                           for row 
                           in np.argsort(np.abs(weights))])
    sorted_weights = np.array([list(wt[index]) 
                               for wt, index 
                               in zip(weights,sorted_indices)])
    sorted_terms = np.array([list(feature_names[row]) 
                             for row 
                             in sorted_indices])
    
    topics = [np.vstack((terms.T, 
                     term_weights.T)).T 
              for terms, term_weights 
              in zip(sorted_terms, sorted_weights)]     
    
    return topics            
  

In [9]:

                       
def print_topics_udf(topics, total_topics=1,
                     weight_threshold=0.0001,
                     display_weights=False,
                     num_terms=None):
    
    for index in range(total_topics):
        topic = topics[index]
        topic = [(term, float(wt))
                 for term, wt in topic]
        topic = [(word, round(wt,2)) 
                 for word, wt in topic 
                 if abs(wt) >= weight_threshold]
                     
        if display_weights:
            print ('Topic #'+str(index+1)+' with weights')
            print (topic[:num_terms] if num_terms else topic)
        else:
            print ('Topic #'+str(index+1)+' without weights')
            tw = [term for term, wt in topic]
            print (tw[:num_terms] if num_terms else tw)
        print ()

In [10]:
topics = get_topics_terms_weights(weights, feature_names)        

In [13]:
topics[0]

array([['dog', '-0.7237079788968448'],
       ['fox', '-0.7237079788968448'],
       ['jump', '-0.4301012144560848'],
       ['smarter', '-0.33636067013751125'],
       ['cat', '-0.33636067013751125'],
       ['quick', '-0.23425077434507793'],
       ['clever', '-0.23425077434507793'],
       ['slow', '-0.23425077434507774'],
       ['lazy', '-0.23425077434507774'],
       ['program', '2.0095334320133576e-16'],
       ['small', '1.0047667160066788e-16'],
       ['programming', '-6.227934847766131e-17'],
       ['language', '-6.227934847766131e-17'],
       ['excellent', '-4.0397983290353874e-17'],
       ['java', '3.960015795038692e-17'],
       ['python', '3.6027395287725075e-17'],
       ['ruby', '-3.545771880776652e-17'],
       ['popular', '-1.0261536110375495e-17']], dtype='<U32')

In [14]:
print_topics_udf(topics=topics, total_topics=total_topics, weight_threshold=0.15, display_weights=False)

Topic #1 without weights
['dog', 'fox', 'jump', 'smarter', 'cat', 'quick', 'clever', 'slow', 'lazy']

Topic #2 without weights
['programming', 'language', 'python', 'java', 'popular', 'ruby', 'excellent', 'program']

